# Surface Classifier

Quantifying the surface characteristics of an organ is essential to understanding how it interacts with other organs and defining its function. This tool will look at one of your image stacks and calculate surface statistics (more information below) and also export an `.obj` file. You can print this `.obj` file out in the maker space if you would like. Possible insights that could be gained from this tool include:
- 3D shape
- Surface texture
- Unique surface features

<div style="display: flex; justify-content: space-around;">
    <img src="https://raw.githubusercontent.com/agadin/QP2_big_data_project_tools/refs/heads/main/img/surface_rotate1.gif" alt="Surface Rotate 1" style="width: 45%;">
    <img src="https://raw.githubusercontent.com/agadin/QP2_big_data_project_tools/refs/heads/main/img/surface_rotate2.gif" alt="Surface Rotate 2" style="width: 45%;">
</div>

### Outputs:
1. **3D Mesh File**:
   - A `.obj` file representing the 3D reconstructed mesh.
   - Contains vertices and faces to represent the 3D structure.

2. **Surface Properties**:
   The script analyzes the generated `.obj` file and outputs the following metrics:
   - **Surface Area (mm²)**: Total surface area of the mesh.
   - **Volume (mL)**: The internal volume enclosed by the mesh.
   - **Bounding Box Dimensions (mm)**: The dimensions of the axis-aligned bounding box (`x`, `y`, and `z`).
   - **Number of Triangles**: The count of triangular faces in the mesh.
   - **Resolution Metrics**:
     - Average edge length (mm).
     - Maximum edge length (mm).
     - Minimum edge length (mm).
   - **Defect Detection**:
     - Euler Number: A topological measure for connectivity.
     - Non-Manifold Edges: The count of edges that belong to more than two faces, indicating defects.

### Workflow:
1. **Load Images**: Reads all 2D slices from the specified folder and processes them into a 3D binary stack.
2. **Reconstruct Mesh**: Converts the binary stack into a 3D mesh using the marching cubes algorithm.
3. **Save Mesh**: Exports the reconstructed mesh as a `.obj` file.
4. **Analyze Mesh**: Calculates and prints various properties of the mesh for quantifying quality and resolution.

In [3]:
!pip install numpy opencv-python trimesh scikit-image ipython ipywidgets

current_path = '/Users/colehanan/Desktop/group_10/MRI_4' # Change this to the path of the folder containing the image stack


# Calculate the surface properties
Run the following cell to calculate the surface properties of the mesh.

In [5]:
import os
import numpy as np
import cv2
import trimesh
from skimage import measure
from skimage.color import rgb2gray
import tifffile
from IPython.display import display
import trimesh.viewer.notebook

def load_image_stack(folder_path):
    # Get all image files with .png, .jpg, or .tif extensions
    image_files = sorted([f for f in os.listdir(folder_path) if f.lower().endswith(('.png', '.jpg', '.tif'))])
    stack = []
    for image_file in image_files:
        file_path = os.path.join(folder_path, image_file)
        if image_file.lower().endswith('.tif'):
            # Use tifffile for TIFF images
            image = tifffile.imread(file_path)
            # If the TIFF is multichannel (e.g., RGB or RGBA), convert to grayscale
            if image.ndim == 3:
                if image.shape[-1] in [3, 4]:
                    image = rgb2gray(image)
                    # Scale the image back to 0-255 and cast to uint8
                    image = (image * 255).astype(np.uint8)
                else:
                    # If the third dimension isn’t color channels, take the first slice
                    image = image[:, :, 0]
            # Normalize or convert image to uint8 if necessary
            if image.dtype != np.uint8:
                image = cv2.normalize(image, None, 0, 255, cv2.NORM_MINMAX).astype(np.uint8)
        else:
            # For png/jpg, use OpenCV
            image = cv2.imread(file_path, cv2.IMREAD_GRAYSCALE)
        if image is not None:
            stack.append(image)
    if not stack:
        raise ValueError("No images found in folder: " + folder_path)
    return np.array(stack)

def create_mesh_from_stack(image_stack, folder_path):
    if image_stack.ndim != 3:
        raise ValueError("Loaded image stack is not a 3D numpy array. Got shape: " + str(image_stack.shape))
    binary_stack = (image_stack > 0).astype(np.uint8)
    print(folder_path)
    spacing = [4, 10 / 17.53, 10 / 17.53] if "CT" in folder_path else [1, 10 / 17.53, 10 / 17.53]
    print(spacing)
    verts, faces, _, _ = measure.marching_cubes(binary_stack, level=0.5, spacing=spacing)
    mesh = trimesh.Trimesh(vertices=verts, faces=faces)
    return mesh

def save_mesh(mesh, output_path):
    if os.path.exists(output_path):
        os.remove(output_path)
    mesh.export(output_path)

def load_mesh_from_obj(file_path):
    if not os.path.exists(file_path):
        raise FileNotFoundError(f"The file {file_path} does not exist.")
    return trimesh.load(file_path)

def analyze_surface_properties_from_obj(obj_path):
    mesh = load_mesh_from_obj(obj_path)

    volume_ml = abs(mesh.volume) / 1000

    bounding_box = mesh.bounds
    bounding_box_dims = {
        'x_dim_mm': bounding_box[1][0] - bounding_box[0][0],
        'y_dim_mm': bounding_box[1][1] - bounding_box[0][1],
        'z_dim_mm': bounding_box[1][2] - bounding_box[0][2]
    }

    avg_edge_length = np.mean(mesh.edges_unique_length)
    max_edge_length = np.max(mesh.edges_unique_length)
    min_edge_length = np.min(mesh.edges_unique_length)

    euler_number = mesh.euler_number
    non_manifold_edges = mesh.edges_unique.shape[0] - mesh.edges.shape[0]

    properties = {
        'surface_area_mm2': mesh.area,
        'volume_ml': volume_ml,
        'bounding_box_dims_mm': bounding_box_dims,
        'num_triangles': len(mesh.faces),
        'avg_edge_length_mm': avg_edge_length,
        'max_edge_length_mm': max_edge_length,
        'min_edge_length_mm': min_edge_length,
        'euler_number': euler_number,
        'non_manifold_edges': non_manifold_edges
    }
    return properties

def main():
    folder_path = current_path  # Ensure current_path is defined with your folder location
    output_path = "output_mesh.obj"  # Change as needed

    image_stack = load_image_stack(folder_path)
    mesh = create_mesh_from_stack(image_stack, folder_path)
    save_mesh(mesh, output_path)
    print(f"Mesh saved to {output_path}")

    properties = analyze_surface_properties_from_obj(output_path)
    print("Surface Properties from OBJ:", properties)

    scene = trimesh.Scene(mesh)
    display(scene.show())

if __name__ == "__main__":
    main()


/Users/colehanan/Desktop/group_10/MRI_4
[1, 0.5704506560182544, 0.5704506560182544]
Mesh saved to output_mesh.obj
Surface Properties from OBJ: {'surface_area_mm2': 14598.340350662704, 'volume_ml': 73.3337829467512, 'bounding_box_dims_mm': {'x_dim_mm': 119.0, 'y_dim_mm': 65.03137479, 'z_dim_mm': 53.05191101}, 'num_triangles': 78270, 'avg_edge_length_mm': 0.7362917990258008, 'max_edge_length_mm': 1.1512662400567657, 'min_edge_length_mm': 0.40336951587619896, 'euler_number': 32, 'non_manifold_edges': -117405}
